### Debugging with the Unittest module
#### Robert Palmere
#### Email: rdp135@chem.rutgers.edu

<< Unit testing explanation >>

First we write a program. Here we will generate a calculator. We will refrain from using external libraries to demonstrate the applicability of the unittest module in debugging our code.

In this example, we consider evaluating the expression: 1+(2+2+(2+2)) using a calculator which accepts this expression as user input. We then debug this class with unit testing to ensure that the class is defined sufficiently.

In [25]:
# We will refrain from using external libraries which can provide warnings / error messages

class Calculator(object):
    """ Class describing a calculator
    """
    
    def __init__(self, in_):
        self.input = in_
        self.current_value = 0
        self.current_operation = None
        self.operation_log = []
        self.value_log = []
        self.operation_indices = []
        self.value_indices = []
        
        self.operators = {
            'add' : '+',
            'subtract' : '-',
            'multiply' : '*',
            'divide' : '/',
            'quantity_l' : '(',
            'quantity_r' : ')'
        }
        
    def call(self, name, v1, v2):
        f = getattr(self, name)
        return f(v1, v2)
    
    def scan(self):
        """ Scan for operators and values in expression and append logs
        """
        chars = self.input.lower()
        for n, c in enumerate(chars):
            if c in self.operators.values():
                self.operation_log.append(c)
                self.operation_indices.append(n)
            else:
                try:
                    c = float(c)
                    self.value_log.append(c)
                    self.value_indices.append(n)
                except:
                    raise ValueError(f'Could not convert {c} to float.')
                    
    def quantities(self):
        """ Determine how many quantities ( '()' ) are in the expression and parse into subexpressions
        """ 
        
        # Determine how many quantities
        
        ql, qr = 0, 0

        il, ir = [], []
        
        for loc, op in enumerate(self.input):
            if op == self.operators['quantity_l']:
                ql += 1
                il.append(loc)
            if op == self.operators['quantity_r']:
                qr += 1
                ir.append(loc)
                
                
        if ql != qr:
            raise ValueError("Parentheses of the input expression must be provided in pairs.")
        
        # Parse into subexpressions and return
        
        ilr = [i for i in reversed(il)]
        spans = list(zip(ilr, ir))
        
        # Capture everything else that is not within a quantity
        
        largest_span = (0, 0)
        for n, span in enumerate(spans):
            delta = abs(span[0]-span[1])
            if delta > abs(largest_span[0]-largest_span[1]):
                largest_span = span
        
        remaining_span = tuple( set([n for n, char in enumerate(self.input)]) - set([i for i in range(largest_span[0], largest_span[1]+1)]) )
        
        return spans, remaining_span

        
    def calculate(self):
        """ Calculate the result from user input following order of operations
        """
        
        k = list(self.operators.keys())
        v = list(self.operators.values())
        
        # Unpack quantities

        quantities, remainder = self.quantities()
        
        # Gather operations that can be performed

        actions = [i for i in self.operators.values() if i not in ['(', ')']]

        # Locate the values from the user input
        
        value_indxs = []
        for q in quantities:
            indices = [i for i in range(q[0], q[0]+abs(q[1]-q[0]))]
            v_indices = [i for i in indices if self.input[i] not in self.operators.values()]
            value_indxs.append(v_indices)
        
        # Remove repeat indices
        
        for n, sublist in enumerate(value_indxs):
            for r in range(n+1, len(value_indxs)):
                delta = set(value_indxs[r]) - set(value_indxs[n])
                value_indxs[r] = list(delta)
                
        # Prepend values not part of a quantity
        
        remaining = []
        for ele in remainder:
            if self.input[ele] in self.operators.values():
                pass
            else:
                remaining.append(ele)

        value_indxs.append(remaining)
                
        # Compute quantity values            
        
        q_results = []
        for n, q in enumerate(value_indxs):
            if len(q) > 1:
                for i in range(q[0], q[0]+abs(q[0]-q[1])):
                    char = self.input[i]
                    if char in actions:

                        # Call the function

                        function_name = k[v.index(self.input[i])]

                        if len(value_indxs[n]) > 1:
                            quantity_result = self.call(function_name, float(self.input[value_indxs[n][0]]), float(self.input[value_indxs[n][1]]))
                            q_results.append(quantity_result)
            else:
                for i in q:
                    q_results.append(float(self.input[i]))
                        
        # Now that we have the quantity results, we need to know the operators between all quantities and then execute those operations
        # We do this by going to operator log and if the index is within the span of a quantity it is ignored, otherwise it is counted
        
        o = list(zip(self.operation_log, self.operation_indices))
        
        # Ensure that none of the expected indices of operators are in the range of quantities
        
        not_allowed = []
        for span in value_indxs:
            if len(span) > 1:
                for op in o:
                    if op[1] in range(span[0], span[1]):
                        not_allowed.append(op[1])
        
        final_operations = []
        for op in o:
            if op[1] not in not_allowed and op[0] in actions:
                final_operations.append(op)
                
        # Check that the final number of operations is n-1 of the final quantities
        
        if len(q_results) != len(final_operations)-1:
            pass
        else:
            raise ValueError('Final number of operations is not 1 less than quantity results.')
        
        # Perform the final operations to acquire the result
        
        final_result = 0
        for n, f in enumerate(final_operations):

            function_name = k[v.index(f[0])]
            
            if len(q_results) > 1:
                if n == 0:
                    final_result += self.call(function_name, q_results[n], q_results[n+1])
                else:
                    final_result = self.call(function_name, final_result, q_results[n+1])
           
            # We have not handled inputs that do not include quantities here which we will address with unit testing 

        return final_result
    
    @staticmethod
    def add(a, b):
        return a+b
    
    def subtract(self, a, b):
        return a-b
    
    def multiply(self, a, b):
        return a*b
    
    def divide(self, a, b):
        return a/b
    
    def square(self, a, b):
        return a**2
    
    def power(self, a, b):
        return a**b
    
    def clear(self):
        self.value_log = []
        self.operation_log = []
        self.current_value = 0

In [26]:
c = Calculator('1+(2+2+(2+2))')
c.scan()
result = c.calculate()
print(result)

9.0


We seem to get the desired result in our single test case. Now we:

1. Test the code using various test cases (i.e. what happens when a user enters letters?)
2. Modify the class perhaps by adding a new method or changing syntax
3. Test the code again using the unittest case we developed.

This way we enable rapid testing and assurance that we have a rubric to follow to ensure our code works as we continue to work on it

We want to first generate a class that will test our Calculator class:

In [27]:
import unittest

class Test(unittest.TestCase):
    '''Class for unit testing Calculator'''
    
    def test_add(self): # "test_" is the required prefix for the base class 'TestCase'
        self.assertEqual(Calculator.add(-1, 1), 0)
        self.assertEqual(Calculator.add(1, 1), 0)

In [28]:
Test().test_add()

AssertionError: 2 != 0

Directly calling the function from our TestCase class is one method for unit testing, but does not take full advantage of the unittest module.\

Once we inherit from unittest.TestCase, methods with the 'test_' prefix will update TestCase attributes associated with the unittest module. Working through the unittest can be automated (rather than calling each test method) using the 'main()' method of the unittest module.

In [24]:
unittest.main()

E
ERROR: /Users/rdp135/Library/Jupyter/runtime/kernel-856bab1e-03be-4489-b6d8-e087cce79983 (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/Users/rdp135/Library/Jupyter/runtime/kernel-856bab1e-03be-4489-b6d8-e087cce79983'

----------------------------------------------------------------------
Ran 1 test in 0.011s

FAILED (errors=1)


SystemExit: True

/Users/rdp135/opt/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


We now move to test the calculate() method which we know does not consider entries outside of quantities.

In [95]:
class TestCalculator(unittest.TestCase):
    '''Class for unit testing our calculator'''
    
    def __init__(self):
        super().__init__()
        self.test_inputs = {'1+(2+2+(2+2))' : 'Nested Quantity Addition with Non-quantity',
                            '(1+1)' : 'Single Quantity Addition',
                            '1*1' : 'Non-quantity Multiplication'
                           }
        self.expected_results = (9.0, 2, 1.0)
        if len(self.test_inputs) != len(self.expected_results):
            raise ValueError('Test input definitions must match length of the expected results tuple')
    
    def test_calculate(self):
        for n, i in enumerate(self.test_inputs):
            print(self.test_inputs[i])
            test = Calculator(i)
            test.scan()
            test_value = test.calculate()

            self.assertEqual(float(test_value), float(self.expected_results[n]))

In [96]:
TestCalculator().test_calculate()

Nested Quantity Addition with Non-quantity
Single Quantity Addition


AssertionError: 0.0 != 2.0

From this simple test, we were able to identify that there is an error in the calculate() method when computing the expression that is representative of what we labeled as "Single Quantity Addition".

One would need to ensure that, as they write their Test() class for unit testing their other classes / modules, that their tests are sufficiently written to capture (un)expected behavior. For writing more sophisticated tests, the documentation has lists and examples to work from which can be found at https://docs.python.org/3/library/unittest.html. 